Выборка данных у нас уже есть - мы будем использовать данные с сайта kaggle для такой же задачи. \
**Ссылка на соревнование:** https://www.kaggle.com/adammaus/predicting-churn-for-bank-customers

In [1]:
import catboost
print(catboost.__version__)

1.2.7


In [2]:
#Наши стандартные библиотеки
import numpy as np
import pandas as pd

# Для визуализации
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pd.options.display.max_rows = None
pd.options.display.max_columns = None

# Домашнее задание

# Обучил 6 моделей
# 1 - /workspaces/ml_course/Project/data/my_model.pkl - Логистическая регрессия 6 новых признаков 71
# 2 - /workspaces/ml_course/Project/data/my_model.pkl2 - Логистическая регрессия 4 новых признака 72
# 3 - /workspaces/ml_course/Project/data/my_model_v2.pkl - RandomForestRegression 4 новых признака c параметром балансировки классов 72
# 4 - /workspaces/ml_course/Project/data/best_model.pkl - RandomForestRegression с GridSearch 4 новых признака 76
# 5 - /workspaces/ml_course/Project/data/best_model_v2.pkl RandomForestRegression с GridSearch без генерации новых признаков 76
# 6 - /workspaces/ml_course/Project/data/catboost_model.cbm 88

In [5]:
import joblib

model = joblib.load('/workspaces/ml_course/Project/data/best_model_v2.pkl')

In [18]:
# Создал новые признаки на основе существующих
df = pd.read_csv("/workspaces/ml_course/Project/data/customer_churn.csv")
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Age_CreditScore_interaction,Balance_EstimatedSalary_interaction,Age_Activity_interaction,Balance_Exited_interaction
0,461,Spain,Female,25,6,0.00,2,1,1,15306.29,0,11525,0.000000e+00,25,0.0
1,619,France,Female,35,4,90413.12,1,1,1,20555.21,0,21665,1.858461e+09,35,0.0
2,699,France,Female,40,8,122038.34,1,1,0,102085.35,0,27960,1.245833e+10,0,0.0
3,558,Germany,Male,41,2,124227.14,1,1,1,111184.67,0,22878,1.381215e+10,41,0.0
4,638,France,Male,34,5,133501.36,1,0,1,155643.04,0,21692,2.077856e+10,34,0.0


Следующая часть урока полностью ложится на ваши плечи:
* Нужно правильно сгенерировать вспомогательные признаки, но сильно можно не увлекаться.
* Правильно их обработать для подачи в модель.
* И построить модель, которая будет давать достаточное качество.

Для защиты проекта необходимо будет построить модель, качество которой дает ROC-AUC не ниже 0.8. Оцениваться модель будет с помощью бота, а точнее функции - `BotProductStar.production_quality(answer=submission)`



submission - это датафрем, в котором в первой колонке указан RowNumber из файла test, а в колонке predict предсказанная вероятность ухода клиента.

Здесь можно продолжить построение модели. Давайте вспомним, какие пункты осталось сделать.

In [22]:
from catboost import CatBoostClassifier
import pandas as pd

In [23]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [24]:
from teacher_bot.churn_prediction import ChurnPrediction
BotProductStar = ChurnPrediction()
BotProductStar.test_task(df = train)


Привет! Приятно познакомиться!
Думаю...
Ого! Датасет, сейчас мы будем его исследовать, интересно, что в нем.
Сам я не справляюсь - нужна твоя помощь


In [25]:
model = CatBoostClassifier()
model.load_model("/workspaces/ml_course/Project/data/catboost_model.cbm", format='cbm')

test = pd.read_csv('./data/test.csv')

# Определяем категориальные признаки
cat_features = ['Geography', 'Gender']

test[cat_features] = test[cat_features].astype(str)

X_test = test.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
submission = test[['RowNumber']].copy()
submission['predict'] = model.predict_proba(X_test)[:, 1]

# Оцениваем модель 
BotProductStar.production_quality(answer=submission)


Запускаю тестирование...
Проверяю метрики...
Твой результат: 0.8809183242022688
Наконец-то мы  сэкономили денег! Чтобы себя порадовать - можно посчитать примерные значения в деньгах, как мы посчитали с бейзлайном :). Финальная кодовая фраза '- моя любимая наука' Если вспомнить все предыдущие получится: Data Science - моя любимая наукаЧто-то правда, то правда - обожаю анализировать данные, особенно, когда мне помогают
